<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# Trabajo práctico 3: Vectores de palabras

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



**Trabajo práctico porpuesto por:** Julio Waissman Vilanova (julio.waissman@unison.mx)

**Desarrollado por:** _Poner tu nombre y correo electrónico aquí_


En este trabajo práctico vamos a generar y usar un modelo de vectores de inserciones de palabras. Para este proyecto práctico, contamos con un corpus de preguntas que se han hecho en *Stack-overflow* para diferentes lenguajes de programación.

La idea final es contar con una función que, ante una frase, determine si es una pregunta sobre algun lenguaje de programación, que determine el lenguaje de programación, y que los de una recomendación sobre una pregunta similar. Dado que el corpus lo tengo solo en inglés (casi no hay preguntas técnicas en español), vamos a hacer todo en inglés para este ejercicio. Para la detectar si una frase es de interés nuestro, se utiliza un corpus de diálogos de peliculas de cine.

## 1. Obteniendo los documentos

Los documentos se encuentran en dos archivos *picle*, los cuales los precargamos con los módulos necesarios para el trabajo práctico.

In [5]:
import warnings; warnings.simplefilter('ignore')
import re
import numpy as np
import pandas as pd
import sklearn
import gensim

dt_post = pd.read_pickle('datos/preguntas_so.pkl')
dt_dialog = pd.read_pickle('datos/dialogos.pkl')

display(dt_post.tail())
display(dt_dialog.tail())

print("Las etiquetas (lenguajes) existentes son:")
print(list(dt_post['tag'].unique()))

,post_id,title,tag
2171570,45887455,What is the difference between node.js and ayo...,javascript
2171571,45887857,Why do sequential containers have both size_ty...,c_cpp
2171572,45892983,"why 1 + + ""1"" === 2; +""1"" + + ""1"" === 2 and ""1...",javascript
2171573,45893693,Why does the first line work but the second li...,javascript
2171574,45898184,Can I safely convert struct of floats into flo...,c_cpp


,text,tag
218604,Lord Chelmsford seems to want me to stay back ...,dialogue
218605,I'm to take the Sikali with the main column to...,dialogue
218606,"Your orders, Mr Vereker?",dialogue
218607,"Good ones, yes, Mr Vereker. Gentlemen who can ...",dialogue
218608,Colonel Durnford... William Vereker. I hear yo...,dialogue


Las etiquetas (lenguajes) existentes son:
['c#', 'php', 'c_cpp', 'python', 'ruby', 'java', 'javascript', 'vb', 'r', 'swift']


como podemos observar hay más de dos millones de documentos sobre consultas a *stack overflow* 

## 2. Generando un modelo de palabras específico al problema

In [6]:
simbolos_a_espacio_re = re.compile('[/(){}\[\]\|@,;]')
simbolos_a_eliminar_re = re.compile('[^0-9a-z #+_]')
quita_palabras_vacias = gensim.parsing.preprocessing.remove_stopwords

# Función de normalización
def normaliza_texto(texto):
    """
    Normaliza texto y lo divide en tokens (asumiendo el espacio como separación de tokens)
    
    :param texto: Una cadena de caracteres con el texto a normalizar
    
    :return: Una lista de tokens (strings)
    
    """
    texto = texto.lower()
    texto = simbolos_a_espacio_re.sub(' ', texto)
    texto = simbolos_a_eliminar_re.sub('', texto)
    texto = quita_palabras_vacias(texto)
    return texto

In [7]:
# Escribe el resultado en un archivo de texto
with open('datos/stackovr_normalizados.txt', 'w', encoding='utf8') as fp:
    for documento in dt_post['title'].values:
        fp.write(normaliza_texto(documento) + '\n')
    for documento in dt_dialog['text'].values:
        fp.write(normaliza_texto(documento) + '\n')

In [12]:
!fasttext skipgram -input datos/stackovr_normalizados.txt -output modelos/fstxt_model -dim 100

Read 15M words
Number of words:  49658
Number of labels: 0


Progress: 2.3%  words/sec/thread: 64762  lr: 0.048872  loss: 2.362507  eta: 0h1m  .0%  words/sec/thread: 2824  lr: 0.049983  loss: 4.122480  eta: 0h37m 0.0%  words/sec/thread: 3408  lr: 0.049979  loss: 4.127660  eta: 0h31m 0.0%  words/sec/thread: 3997  lr: 0.049975  loss: 4.126355  eta: 0h26m 0.1%  words/sec/thread: 4878  lr: 0.049969  loss: 4.125077  eta: 0h21m 0.1%  words/sec/thread: 6863  lr: 0.049956  loss: 4.122373  eta: 0h15m 0.1%  words/sec/thread: 9011  lr: 0.049941  loss: 4.118125  eta: 0h11m 0.1%  words/sec/thread: 9982  lr: 0.049934  loss: 4.113279  eta: 0h10m 0.2%  words/sec/thread: 12163  lr: 0.049918  loss: 4.032672  eta: 0h8m 0.2%  words/sec/thread: 14054  lr: 0.049903  loss: 3.939163  eta: 0h7m 0.2%  words/sec/thread: 14774  lr: 0.049897  loss: 3.868558  eta: 0h7m 0.2%  words/sec/thread: 15903  lr: 0.049888  loss: 3.818021  eta: 0h6m 0.2%  words/sec/thread: 16802  lr: 0.049881  loss: 3.741155  eta: 0h6m 0.3%  words/sec/thread: 18998  lr: 0.049862  loss: 3.592333  eta: 0

Progress: 6.9%  words/sec/thread: 86529  lr: 0.046536  loss: 2.115179  eta: 0h1m 4.6%  words/sec/thread: 78567  lr: 0.047687  loss: 2.188379  eta: 0h1m 4.6%  words/sec/thread: 78519  lr: 0.047685  loss: 2.187979  eta: 0h1m 4.7%  words/sec/thread: 78383  lr: 0.047665  loss: 2.186858  eta: 0h1m 4.7%  words/sec/thread: 78490  lr: 0.047657  loss: 2.186063  eta: 0h1m 4.7%  words/sec/thread: 78551  lr: 0.047650  loss: 2.185640  eta: 0h1m 4.7%  words/sec/thread: 78606  lr: 0.047644  loss: 2.185179  eta: 0h1m 4.7%  words/sec/thread: 78617  lr: 0.047639  loss: 2.184824  eta: 0h1m 4.7%  words/sec/thread: 78656  lr: 0.047633  loss: 2.184250  eta: 0h1m 4.7%  words/sec/thread: 78633  lr: 0.047630  loss: 2.183875  eta: 0h1m 4.8%  words/sec/thread: 78033  lr: 0.047613  loss: 2.182146  eta: 0h1m 4.8%  words/sec/thread: 77535  lr: 0.047602  loss: 2.181099  eta: 0h1m 4.8%  words/sec/thread: 77718  lr: 0.047581  loss: 2.179867  eta: 0h1m 4.9%  words/sec/thread: 77968  lr: 0.047554  loss: 2.177888  eta: 0

Progress: 9.2%  words/sec/thread: 93408  lr: 0.045423  loss: 2.075086  eta: 0h1m 7.0%  words/sec/thread: 86656  lr: 0.046520  loss: 2.114883  eta: 0h1m 7.0%  words/sec/thread: 86716  lr: 0.046512  loss: 2.114522  eta: 0h1m 7.0%  words/sec/thread: 86777  lr: 0.046504  loss: 2.114020  eta: 0h1m 7.0%  words/sec/thread: 86897  lr: 0.046489  loss: 2.113480  eta: 0h1m 7.0%  words/sec/thread: 86962  lr: 0.046481  loss: 2.112958  eta: 0h1m 7.1%  words/sec/thread: 87027  lr: 0.046472  loss: 2.112370  eta: 0h1m 7.1%  words/sec/thread: 87079  lr: 0.046466  loss: 2.111968  eta: 0h1m 7.1%  words/sec/thread: 87139  lr: 0.046458  loss: 2.111595  eta: 0h1m 7.1%  words/sec/thread: 87204  lr: 0.046450  loss: 2.111364  eta: 0h1m 7.1%  words/sec/thread: 87238  lr: 0.046445  loss: 2.111261  eta: 0h1m 7.1%  words/sec/thread: 87306  lr: 0.046434  loss: 2.110779  eta: 0h1m 7.1%  words/sec/thread: 87365  lr: 0.046427  loss: 2.110114  eta: 0h1m 7.2%  words/sec/thread: 87417  lr: 0.046419  loss: 2.109590  eta: 0

Progress: 11.7%  words/sec/thread: 98429  lr: 0.044161  loss: 2.043195  eta: 0h0m .2%  words/sec/thread: 93508  lr: 0.045399  loss: 2.074522  eta: 0h1m 9.2%  words/sec/thread: 93540  lr: 0.045392  loss: 2.074343  eta: 0h1m 9.2%  words/sec/thread: 93592  lr: 0.045383  loss: 2.073921  eta: 0h1m 9.2%  words/sec/thread: 93635  lr: 0.045375  loss: 2.073622  eta: 0h1m 9.3%  words/sec/thread: 93695  lr: 0.045362  loss: 2.073204  eta: 0h1m 9.3%  words/sec/thread: 93750  lr: 0.045353  loss: 2.072946  eta: 0h1m 9.3%  words/sec/thread: 93794  lr: 0.045344  loss: 2.072703  eta: 0h1m 9.3%  words/sec/thread: 93818  lr: 0.045337  loss: 2.072440  eta: 0h1m 9.3%  words/sec/thread: 93860  lr: 0.045329  loss: 2.071988  eta: 0h1m 9.4%  words/sec/thread: 93903  lr: 0.045322  loss: 2.071482  eta: 0h1m %  words/sec/thread: 93927  lr: 0.045315  loss: 2.071259  eta: 0h1m 9.4%  words/sec/thread: 93962  lr: 0.045308  loss: 2.071014  eta: 0h1m 9.4%  words/sec/thread: 94005  lr: 0.045299  loss: 2.070821  eta: 0h1m

...

Progress: 91.6%  words/sec/thread: 114133  lr: 0.004209  loss: 1.838989  eta: 0h0m 89.2%  words/sec/thread: 113914  lr: 0.005417  loss: 1.842194  eta: 0h0m 89.2%  words/sec/thread: 113915  lr: 0.005409  loss: 1.842178  eta: 0h0m 89.2%  words/sec/thread: 113918  lr: 0.005402  loss: 1.842131  eta: 0h0m 89.2%  words/sec/thread: 113922  lr: 0.005386  loss: 1.842106  eta: 0h0m 89.3%  words/sec/thread: 113927  lr: 0.005369  loss: 1.842065  eta: 0h0m 89.3%  words/sec/thread: 113923  lr: 0.005348  loss: 1.842022  eta: 0h0m 89.3%  words/sec/thread: 113920  lr: 0.005338  loss: 1.841997  eta: 0h0m 89.3%  words/sec/thread: 113916  lr: 0.005329  loss: 1.841970  eta: 0h0m 0.005318  loss: 1.841957  eta: 0h0m 89.4%  words/sec/thread: 113921  lr: 0.005311  loss: 1.841944  eta: 0h0m 89.4%  words/sec/thread: 113923  lr: 0.005302  loss: 1.841929  eta: 0h0m 89.4%  words/sec/thread: 113922  lr: 0.005297  loss: 1.841907  eta: 0h0m 89.4%  words/sec/thread: 113925  lr: 0.005289  loss: 1.841901  eta: 0h0m 89.4%

Progress: 94.0%  words/sec/thread: 114322  lr: 0.003009  loss: 1.835868  eta: 0h0m 91.6%  words/sec/thread: 114137  lr: 0.004195  loss: 1.838960  eta: 0h0m 91.6%  words/sec/thread: 114141  lr: 0.004183  loss: 1.838936  eta: 0h0m 91.7%  words/sec/thread: 114145  lr: 0.004167  loss: 1.838899  eta: 0h0m 91.7%  words/sec/thread: 114146  lr: 0.004161  loss: 1.838878  eta: 0h0m 91.7%  words/sec/thread: 114148  lr: 0.004154  loss: 1.838851  eta: 0h0m 91.7%  words/sec/thread: 114150  lr: 0.004144  loss: 1.838806  eta: 0h0m 91.7%  words/sec/thread: 114150  lr: 0.004136  loss: 1.838786  eta: 0h0m 91.7%  words/sec/thread: 114153  lr: 0.004128  loss: 1.838758  eta: 0h0m 91.8%  words/sec/thread: 114156  lr: 0.004120  loss: 1.838733  eta: 0h0m 91.8%  words/sec/thread: 114157  lr: 0.004113  loss: 1.838719  eta: 0h0m 91.8%  words/sec/thread: 114159  lr: 0.004106  loss: 1.838681  eta: 0h0m 91.8%  words/sec/thread: 114160  lr: 0.004096  loss: 1.838658  eta: 0h0m 91.8%  words/sec/thread: 114162  lr: 0.00

Progress: 96.6%  words/sec/thread: 114615  lr: 0.001710  loss: 1.832485  eta: 0h0m 94.0%  words/sec/thread: 114326  lr: 0.002991  loss: 1.835816  eta: 0h0m 94.0%  words/sec/thread: 114327  lr: 0.002983  loss: 1.835805  eta: 0h0m 94.1%  words/sec/thread: 114329  lr: 0.002970  loss: 1.835761  eta: 0h0m 94.1%  words/sec/thread: 114333  lr: 0.002954  loss: 1.835709  eta: 0h0m 94.1%  words/sec/thread: 114340  lr: 0.002927  loss: 1.835668  eta: 0h0m 94.2%  words/sec/thread: 114345  lr: 0.002911  loss: 1.835624  eta: 0h0m 94.2%  words/sec/thread: 114346  lr: 0.002902  loss: 1.835621  eta: 0h0m 94.2%  words/sec/thread: 114347  lr: 0.002881  loss: 1.835569  eta: 0h0m 94.3%  words/sec/thread: 114348  lr: 0.002873  loss: 1.835540  eta: 0h0m 94.3%  words/sec/thread: 114351  lr: 0.002866  loss: 1.835529  eta: 0h0m 94.3%  words/sec/thread: 114352  lr: 0.002857  loss: 1.835509  eta: 0h0m 94.3%  words/sec/thread: 114354  lr: 0.002848  loss: 1.835503  eta: 0h0m 94.3%  words/sec/thread: 114356  lr: 0.00

Progress: 99.4%  words/sec/thread: 114846  lr: 0.000299  loss: 1.828905  eta: 0h0m 96.6%  words/sec/thread: 114616  lr: 0.001685  loss: 1.832403  eta: 0h0m 96.7%  words/sec/thread: 114620  lr: 0.001669  loss: 1.832342  eta: 0h0m 96.7%  words/sec/thread: 114623  lr: 0.001653  loss: 1.832303  eta: 0h0m 96.7%  words/sec/thread: 114625  lr: 0.001644  loss: 1.832273  eta: 0h0m 96.7%  words/sec/thread: 114630  lr: 0.001627  loss: 1.832227  eta: 0h0m 96.8%  words/sec/thread: 114634  lr: 0.001605  loss: 1.832174  eta: 0h0m 96.8%  words/sec/thread: 114637  lr: 0.001596  loss: 1.832145  eta: 0h0m %  words/sec/thread: 114638  lr: 0.001588  loss: 1.832137  eta: 0h0m 96.8%  words/sec/thread: 114637  lr: 0.001580  loss: 1.832116  eta: 0h0m 96.9%  words/sec/thread: 114640  lr: 0.001572  loss: 1.832107  eta: 0h0m 96.9%  words/sec/thread: 114641  lr: 0.001563  loss: 1.832099  eta: 0h0m 96.9%  words/sec/thread: 114647  lr: 0.001540  loss: 1.832016  eta: 0h0m 96.9%  words/sec/thread: 114647  lr: 0.001531

Progress: 100.0%  words/sec/thread: 114890  lr: 0.000000  loss: 1.828100  eta: 0h0m 9.4%  words/sec/thread: 114850  lr: 0.000275  loss: 1.828884  eta: 0h0m 99.5%  words/sec/thread: 114852  lr: 0.000266  loss: 1.828835  eta: 0h0m 99.5%  words/sec/thread: 114852  lr: 0.000260  loss: 1.828822  eta: 0h0m 99.5%  words/sec/thread: 114854  lr: 0.000250  loss: 1.828792  eta: 0h0m 99.5%  words/sec/thread: 114857  lr: 0.000242  loss: 1.828765  eta: 0h0m 99.5%  words/sec/thread: 114858  lr: 0.000235  loss: 1.828744  eta: 0h0m 99.5%  words/sec/thread: 114858  lr: 0.000227  loss: 1.828728  eta: 0h0m 99.6%  words/sec/thread: 114860  lr: 0.000219  loss: 1.828717  eta: 0h0m 99.6%  words/sec/thread: 114863  lr: 0.000203  loss: 1.828666  eta: 0h0m 99.6%  words/sec/thread: 114867  lr: 0.000178  loss: 1.828593  eta: 0h0m 99.7%  words/sec/thread: 114867  lr: 0.000172  loss: 1.828572  eta: 0h0m 99.7%  words/sec/thread: 114868  lr: 0.000158  loss: 1.828502  eta: 0h0m 99.7%  words/sec/thread: 114871  lr: 0.00

In [8]:
modelo = gensim.models.FastText.load_fasttext_format('modelos/fstxt_model')

In [10]:
modelo.wv.most_similar(positive=['king', 'women'], negative=['man'], topn=5)

[('kings', 0.7780598402023315),
 ('womens', 0.7381753325462341),
 ('kingdoms', 0.7262006402015686),
 ('dancers', 0.7194182276725769),
 ('porn', 0.7169497609138489)]

In [11]:
modelo.wv.most_similar(positive=['dictionary', 'java'], negative=['python'], topn=5)

[('hashmap', 0.7939369082450867),
 ('arraylistnamevaluepair', 0.7892711162567139),
 ('arraylisthashmap', 0.7810637950897217),
 ('arraylistcustomobject', 0.7795299291610718),
 ('arrayliststring', 0.776357889175415)]

In [12]:
tags = list(dt_post['tag'].unique())
tags_vec = np.zeros((len(tags), modelo.vector_size))
for (i, tag) in enumerate(tags):
    tags_vec[i,:] = modelo[tag]

palabra = 'c#'
sims = modelo.wv.cosine_similarities(modelo[palabra], tags_vec)

print("Similaridad de los 'tags' con la palabra " + palabra)
for (tag, sim) in zip(tags, sims):
    print("{:15}{}".format(tag, sim))
    


Similaridad de los 'tags' con la palabra c#
c#             0.999999978515174
php            0.5179184861711419
c_cpp          0.37267283266567763
python         0.49720232778872786
ruby           0.4634916895670994
java           0.6097577125740264
javascript     0.5787009937733788
vb             0.8911569775581926
r              0.41652310651539354
swift          0.4199511596253413


## 3. Generando un discriminador de pregunta técnica vs texto normal con el modelo

In [13]:
def doc_a_vec(doc, modelo):
    """
    Convierte un documento en vector utilizando
    el viejo truco de sacar la media de las palabras
    que existen en el vocabulario.
    
    :param doc: Una lista con las palabras del documento
    :param modelo: Un modelo tipo FastText o cualquiero otro modelo de vectores
    
    :return un vector que representa un documento
    
    """
    return modelo[doc].mean(axis=0)

In [63]:
num_muestras = dt_dialog.shape[0]
positivos = dt_post['title'].sample(num_muestras, random_state=10).values
negativos = dt_dialog['text'].values

x_texto = np.r_[positivos, negativos]
y = np.r_[np.ones(num_muestras), -1 * np.ones(num_muestras)]

In [76]:
x = np.zeros((2 * num_muestras, modelo.vector_size))
for (i, doc) in enumerate(x_texto):
    if doc:
        doc = normaliza_texto(doc)
        print("{}-{}".format(i, doc))
        x[i, :] = doc_a_vec(doc.split(), modelo)
    
x.shape

0-preg_replace replace match
1-wordpress suitable large database querying
2-ruby compiling dynamic applications windows
3-write json file jquery ajax php
4-java validation code check wether given input number word date
5-fputcsv putting duplicate values file
6-use singleton design pattern real time java applications
7-dropdown list error viewdata item type ienumerableselectlistitem
8-redirect previous page filled input php
9-add widget panel position
10-adding li element li tag
11-afnetworking httpclient subclass xmlparser
12-scrape specific text html tags occur outside tags
13-xcode importing c++ project problems referenced
14-remove textshadow column headers gwt datagrid
15-method taskscheduler
16-deploying ruby rails app cpanel ssh connection
17-run foreach codes output
18-rgb hex conversion
19-current displaymode mobile mvc4
20-android jsonarray arraylist
21-error creating effect memory c++ directx 11
22-json data newbie
23-reset camera position rotation trackballcontrols
24-combin

709-c++ simulate left mouse click minimized program
710-build nested list jquery header selector
711-collapse columns matrix column
712-django form skip model validation
713-split date c#
714-compare values single hashmap collections
715-connect mongodb plugin maven
716-undefined method match nil class oracle enhanced adapter connection
717-conditional imports swift
718-strcpy arrays structs elements doesnt work
719-apply condition jframe constructor
720-prevent encoding existing html entities convert amp amp amp amp
721-unobtrusive javascript rich text editor
722-rails error param missing value buyer
723-systemruntimeinteropservicessehexception unityenginedll
724-python csv write module need output different rows instead columns
725-textareas text gets erased safari jquery keyup
726-nth root number ruby
727-sound playing android icecream sandwich
728-day current month javascript
729-c# deserialize json
730-setting jackson 2 jersey tomcat
731-mysql result count
732-need help finding ma

1217-java validators
1218-elements canvas
1219-binding button class method py kv file
1220-storyboard expandable textview
1221-magento custom search form
1222-remove characters string
1223-data window wpf mvvm pattern
1224-dimensions video file
1225-heap implementation c++
1226-scipysignalfftconvolve axis
1227-auto upload image page refresh jquery jsp
1228-hide option dropdown 2 selected dropdown 1
1229-c runtime error r6016 space thread data
1230-program crashes running debugging
1231-wordpress wp_editor updating textarea _post data
1232-multiple interface java having method varable
1233-handle multiple clients jsp
1234-add sql server database project setup
1235-servlet security question j_security_check j_username j_password
1236-phonegap camera loading image img tag
1237-razor default dropdown box
1238-add metadata objectivec methods properties
1239-return 302 redirect web service environment
1240-php version adoption stats
1241-check list containing nan
1242-matplotlib draw sequenc

1732-splitting datatable multiple datatables
1733-install php 54 debian 6 dotdeb
1734-keyboard appears view user interaction allowed
1735-display new root category default category magento
1736-activerecord dirty unsaved relations
1737-run arbitrary server software eclipse
1738-javafx smooth animation
1739-fade text changing react
1740-vaadin caption invisible
1741-problems running java program command line interface
1742-create file server simple php html page
1743-fatal error member function fetch nonobject
1744-create pdf print visual basic 2010
1745-webservicerequest array
1746-making ajax aspnet jquery
1747-webclient table
1748-downloading zip files httpget nodejs
1749-error javasqltimestamp cast javasqldate jfreechart
1750-wrapping c++ library objectivec hiding c++ symbols
1751-foreach loop executing called ajax
1752-threading windows service
1753-use windowexternalnotify uglifyjs compressed
1754-create c# bindings complex c++ classes
1755-laravel 5 library load css javascript
17

2182-detect chrome panels enabled
2183-basic backgroundworker usage parameters
2184-filling html form c#
2185-test params passed controller rails 3 rspec
2186-queuesynchronized faster lock
2187-javascript windowlocationhref refreshes page instead redirecting
2188-file cache gae php
2189-whats wrong requested operation performed file usermapped section open
2190-sqlite python error
2191-gradle dependencies file build gradle
2192-transfer value javascript php variable checked radio button different page
2193-ruby handle http runs proxy proxy
2194-single object repository pattern
2195-messageselecteddate value valid
2196-problem nested loops
2197-javalangnoclassdeffounderror initialize class orghibernatecfgannotationconfiguration
2198-ipad crashes try load admob ad
2199-django rest framework 24 children parent detail_route
2200-issue extracting hyperlinks pdf itextsharp
2201-ajax redirect php post return true error
2202-sqlite sql error missing database objectivec
2203-remove vector eleme

...

7073-jquery random number guessing game loop woes
7074-table cell value php post
7075-prepopulate model value mvc3 page
7076-usb connection delegate swift
7077-remove content magento sidebar
7078-creating customattributebuilder nullable property
7079-previouspagefindcontrol getting variable previous page
7080-close tab code
7081-efficient easy method validate text boxes
7082-missing properties httpwebrequest
7083-typeid reinterpret_cast typeid variable
7084-python multiprocessing pool memory leak
7085-python error escape socketerror errno 54 connection reset peer
7086-rails query array attribute contains overlap array
7087-change label background color wpf form clicking button winforms form
7088-auto reload div checkbox form change
7089-adding text expressions r plot
7090-advantages tensorflow numpy general computation
7091-wordpress php the_content read link
7092-js function defined babel script html
7093-spring boot issue ddlauto
7094-export jar file project uses gson jar
7095-filter

7419-selected value htmlradiobuttonfor jquery
7420-operator c++
7421-php array indexing array index vs array index vs array index
7422-c++ force complete template compilation msvc g++
7423-control loop gif xaml
7424-split string bullets types regex java
7425-apache poi remove cthyperlink low level code
7426-python solve 5th order polynomial
7427-dynamically running clamavs clamscan file uploads php
7428-displaying multiple google charts page
7429-pass ints constructor jvalue array
7430-maximum value listt net 20
7431-selection position selected text div javascript
7432-shiny renderdatatable limit text size displayed
7433-trouble finding new line character string reading text txt file
7434-inserting linebreak title attribute javascript jquery
7435-getting remove type arguments error liststring list1 new arrayliststring
7436-converting remote relative paths absolute paths
7437-pandas series frame quantile function taking multiple probabilities
7438-ios swift loading custom table cells
74

7817-jquery height functioning correctly
7818-jquery validation preventing form submission
7819-python twisted wait variable filled event
7820-replace character html tag regex
7821-c# webapi garbage collection
7822-to_json skips attributes nil value
7823-globals stored nodejs windowlike object node
7824-php jquery ajax form submit response
7825-array subscript integer c
7826-produce matplotlib plot xaxis order
7827-stringutils resolved
7828-libgdx wont draw sprite animation
7829-monitoring serverside process rails application ajax xmlhttprequest
7830-import ios contacts tableview
7831-modulus django template
7832-select uitableviews row clicking uiswitch
7833-email id facebook graph api
7834-c rand random
7835-c programming multiple words string
7836-monodevelop set application output fixedwidth font
7837-data jquery ajax
7838-c convert char hex
7839-pytest hide stacktrace lines unittest module
7840-bootstrapsass color codes
7841-java niofloatbuffer serialization
7842-c# parse string 0

8121-questions unix code
8122-creating win32api dll callable language
8123-formatting date php inserting line break
8124-scanf s issue eof
8125-int nsdata
8126-permission denied kivy
8127-dismiss mpmovieplayercontroller screen touch
8128-column belong table multi word query
8129-regex optimisation escaping ampersands java
8130-dynamic projections linqtoentities
8131-kendo ui batch editing row copied different grid results rows grids updated
8132-running tests parallel junit
8133-entity framework 7 create inmemory implementation dbset
8134-undefined method strip deploying sinatra application capistrano
8135-list outlook folders subfolders
8136-php script works fine manually cronjob
8137-generate dynamic audio signal
8138-python beautifulsoup siblings
8139-errors c2059 c2061 c language
8140-php registered functions xslt1 return xml fragment php
8141-load relationships model
8142-connect local mysql server socket issue
8143-collection_select different models value text methods
8144-toploc

ValueError: need at least one array to concatenate

## 4. Generando un clasificador de tópicos con el modelo

In [74]:
modelo["python error unsupported operand type s + int str".split()].mean(axis=0)

array([-0.09049639,  0.24661271, -0.17591703, -0.03256614, -0.01266338,
       -0.0224597 ,  0.339206  , -0.29855677,  0.22176653,  0.15742226,
       -0.1346316 , -0.29670784,  0.05631329,  0.27182448, -0.14735907,
       -0.16909237,  0.3030882 , -0.19962192, -0.02660725, -0.2801763 ,
        0.06611028,  0.0589049 , -0.41912496,  0.12442127,  0.4631061 ,
       -0.0863012 , -0.11674944,  0.03452254, -0.56300044,  0.22812197,
        0.01923675, -0.59961975, -0.12135287, -0.5822334 ,  0.03271319,
       -0.16470003, -0.2445301 ,  0.13734013,  0.04095062,  0.0646313 ,
        0.442553  ,  0.29200363, -0.02484276,  0.00555229,  0.89788926,
       -0.19364357, -0.1258021 , -0.31848162, -0.7691653 , -0.04620411,
       -0.10042325,  0.28020185, -0.08466545,  0.05805634, -0.20625383,
       -0.47976363, -0.09082787,  0.09262765, -0.04248796,  0.01147545,
        0.04151448,  0.46215677,  0.2380473 ,  0.20896631,  0.16755006,
       -0.28455272, -0.14680031, -0.00877271,  0.06928954, -0.15

## 5. Construyendo un recomendador a partir del modelo

## 6. Poniendo todo junto